# Import packages

In [1]:
import pandas as pd
import numpy as np
import os
from anndata import read_h5ad
from sklearn.neighbors import NearestNeighbors


import warnings
warnings.filterwarnings('ignore')
import sys

# sys.path.append('/data/yosef2/users/chenling/scVI/')
sys.path.append('../../utils/')
from annotations_utils import *
import scvi
print(scvi.__version__)

import os
os.getcwd()

import logging
import os
import pickle
from MulticoreTSNE import MulticoreTSNE as TSNE

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from hyperopt import hp

from scvi.inference import UnsupervisedTrainer, AlternateSemiSupervisedTrainer, SemiSupervisedTrainer
from scvi.models import VAE, SCANVI
from copy import deepcopy

from scvi.dataset.anndataset import AnnDatasetFromAnnData

import scanpy as sc
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)

from anndata import read_h5ad
from anndata import AnnData

import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
%matplotlib inline

n_epochs = 150
show_plot = True
test_mode = False
use_batches = True
use_cuda = True
lr = 1e-3
retrain=True


sc.logging.print_versions()

[2020-02-28 11:22:20,013] INFO - scvi._settings | Added StreamHandler with custom formatter to 'scvi' logger.


0.5.0
scanpy==1.4.5.post2 anndata==0.6.22.post1 umap==0.3.10 numpy==1.17.2 scipy==1.3.1 pandas==0.25.1 scikit-learn==0.21.3 statsmodels==0.10.1 python-igraph==0.7.1 louvain==0.6.1


# Load data

In [2]:
data_path = '../../data-objs/pilot1/TSP1_objs_browser/'

In [9]:
import glob
obj_list = glob.glob(data_path+'*.h5ad')
obj_list

['../../data-objs/pilot1/TSP1_objs_browser/TSP1_blood_endothelial.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_all_immune.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_all_endothelial.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_lung_epithelial.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_lung_mixed.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_bladder.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_all_mixed.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_blood_mixed.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_pancreas_endothelial.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_muscle_immune.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_lung_stromal.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_blood.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_muscle_endothelial.h5ad',
 '../../data-objs/pilot1/TSP1_objs_browser/TSP1_all_epithelial.h5ad',
 '../../data-objs/pilot1/TSP1_objs_brows

In [8]:
import os
os.listdir(data_path)[:2]

['.ipynb_checkpoints', 'TSP1_blood_endothelial.h5ad']

# Write objs for browser

In [7]:
adata.write('../../data-objs/pilot1/TSP1_objs_browser/TSP1_all.h5ad')

... storing 'compartment_prediction' as categorical
... storing 'method' as categorical
... storing 'subtissue' as categorical
... storing 'tissue' as categorical


## Write compartment objs

In [8]:
for c in list(set(adata.obs['compartment_prediction'])):
    adata_comp = adata[adata.obs['compartment_prediction']==c].copy()
    
    if adata_comp.shape[0]>0:
        sc.pp.pca(adata_comp)
        sc.pp.neighbors(adata_comp, n_neighbors=20, n_pcs=30, use_rep="X_scvi")
        sc.tl.umap(adata_comp)
        sc.tl.leiden(adata_comp)
        sc.tl.tsne(adata_comp, use_rep = 'X_scvi')
        adata_comp.write_h5ad('../../data-objs/pilot1/TSP1_objs_browser/TSP1_all_'+c+'.h5ad')
    else:
        print("there is no cells to put in "+c)

computing PCA with n_comps = 50
    finished (0:00:10)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:09)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:33)
running Leiden clustering
    finished: found 35 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:18)
computing tSNE
    using the 'MulticoreTSNE' package by Ulyanov (2017)
    finished: added
    'X_tsne', tSNE coordinates (adata.obsm) (0:03:15)
computing PCA with n_comps = 50
    finished (0:00:02)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:02)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:17)
running Leiden clustering
    finished: found 26 clusters and added
    'leiden',

## Write tissue objs

In [9]:
for t in list(set(adata.obs['tissue'])):
    adata_tiss = adata[adata.obs['tissue']==t].copy()
    
    if adata_tiss.shape[0]>0:
        sc.pp.pca(adata_tiss)
        sc.pp.neighbors(adata_tiss, n_neighbors=20, n_pcs=30, use_rep="X_scvi")
        sc.tl.umap(adata_tiss)
        sc.tl.leiden(adata_tiss)
        sc.tl.tsne(adata_tiss, use_rep = 'X_scvi')
        adata_tiss.write_h5ad('../../data-objs/pilot1/TSP1_objs_browser/TSP1_'+t+'.h5ad')
    else:
        print("there is no cells to put in "+t)

computing PCA with n_comps = 50
    finished (0:00:07)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:04)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:18)
running Leiden clustering
    finished: found 36 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:06)
computing tSNE
    using the 'MulticoreTSNE' package by Ulyanov (2017)
    finished: added
    'X_tsne', tSNE coordinates (adata.obsm) (0:01:51)
computing PCA with n_comps = 50
    finished (0:00:02)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:01)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:16)
running Leiden clustering
    finished: found 26 clusters and added
    'leiden',

## Write tissue_compartment objs

In [25]:
for t in ['blood','pancreas']:#list(set(adata.obs['tissue'])):
    adata_tiss = adata[adata.obs['tissue']==t].copy()
    if adata_tiss.shape[0]>0:
        
        for c in list(set(adata_tiss.obs['compartment_prediction'])):
            adata_comp = adata_tiss[adata_tiss.obs['compartment_prediction']==c].copy()

            if adata_comp.shape[0]>0:
                
                try:
                    sc.pp.pca(adata_comp)
                except:
                    sc.pp.pca(adata_comp,n_comps=adata_comp.shape[0]-1)
                
                sc.pp.neighbors(adata_comp, n_neighbors=20, n_pcs=30, use_rep="X_scvi")
                
                try:
                    sc.tl.umap(adata_comp)
                    sc.tl.leiden(adata_comp)
                    sc.tl.tsne(adata_comp, use_rep = 'X_scvi')
                    adata_comp.write_h5ad('../../data-objs/pilot1/TSP1_objs_browser/TSP1_'+t+'_'+c+'.h5ad')
                
                except:
                    print("there is no cells to put in "+t+" "+c)
            
            else:
                print("there is no cells to put in "+t+" "+c)
    
    
    else:
        print("there is no cells to put in "+t)

computing PCA with n_comps = 50
    finished (0:00:07)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:04)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:30)
running Leiden clustering
    finished: found 18 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:02)
computing tSNE
    using the 'MulticoreTSNE' package by Ulyanov (2017)
    finished: added
    'X_tsne', tSNE coordinates (adata.obsm) (0:00:46)
computing PCA with n_comps = 50
    finished (0:00:01)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:01)
running Leiden clustering
    finished: found 7 clusters and added
    'leiden', 

# DGE

## DGE per organ per compartment

In [10]:
adata = sc.read_h5ad(obj_list[1])
adata

AnnData object with n_obs × n_vars = 34561 × 21301 
    obs: 'batch_id', 'compartment_prediction', 'leiden', 'log1p_total_counts', 'manual_cell_ontology', 'manual_free_annotation', 'method', 'n_genes', 'pancreas_prediction', 'scANVI_organ_alternate_training_cell_ontology', 'scANVI_organ_alternate_training_free_annotation', 'scANVI_organ_cell_ontology', 'scANVI_organ_compartment_alternate_training_cell_ontology', 'scANVI_organ_compartment_alternate_training_free_annotation', 'scANVI_organ_compartment_cell_ontology', 'scANVI_organ_compartment_free_annotation', 'scANVI_organ_free_annotation', 'subtissue', 'tissue', 'tissue_method'
    var: 'n_cells-0', 'n_cells-1'
    uns: 'Compartment Prediction_colors', 'Pancreas Prediction_colors', 'Smoothed Compartment Prediction_colors', 'leiden', 'neighbors', 'pca', 'smooth_comp2_colors', 'smooth_comp_colors', 'umap'
    obsm: 'X_pca', 'X_scvi', 'X_tsne', 'X_umap'
    varm: 'PCs'

In [ ]:
for obj in obj_list:
    

In [ ]:
def DEbyOrganCompartment(raw, full, label, organs, compartments,  split_subset=None):
    pred_label = np.asarray(raw.obs[label].values)
    pred_celltype, pred_label = np.unique(pred_label, return_inverse=True)
    for organ in np.unique(raw.obs[organs]):
        if split_subset is None: 
            split_subset = np.unique(raw.obs.loc[raw.obs[organs].values == organ, compartments])
        writer = pd.ExcelWriter(organ+'.DE.xlsx', engine='xlsxwriter')
        res = []
        clust = []
        for compartment in split_subset:
            print(compartment)
            subset = np.logical_and(raw.obs[compartments].values == compartment,
                                   raw.obs[organs].values == organ)
            de_res, de_cluster = full.one_vs_all_degenes(cell_labels=pred_label,
                                                         subset=subset)
            filtered = []
            for i, ct in enumerate(pred_celltype[de_cluster]):
                x = de_res[i]
                filt = np.logical_and(np.abs(x['bayes_factor'].values) > 1.3, x['raw_mean1'].values > 0.1)
#                 filt = x['proba_de']>0.90
                filtered.append(x.loc[filt])
#                 res = x.loc[filt]
#                 if len(ct)>31:
#                     ct = ct[:31]
#                 res.to_excel(writer, sheet_name=ct)
#                 res['celltype'] = ct
#             writer.save()
            temp = []
            for i,x in zip(de_cluster, filtered):
                logfc =np.log(x['scale1']/x['scale2']+1)
#                 logfc = x['median']
                clust_id = np.repeat(i, len(logfc))
                y = pd.DataFrame([clust_id, logfc.values, logfc.index], index=['id','logFC','Genenames']).T
                temp.append(y)
            if len(temp)>0:
                res = pd.concat(temp, axis=1)
                res.to_excel(writer, sheet_name=compartment)
        writer.save()